# Функции-генераторы

![common iterators description](../images/generators.png "Logo Title Text 1")

Выражения-генераторы позволяют нам создавать генераторы, не хранящие элементы в памяти, а вычисляющие их при каждом запросе.  
Однако, у выражений есть минус – с помощью них нельзя реализовать сложную логику.  
Для сложного существуют **функции-генераторы**.

```python
def gen_function(*args, **kwargs):
    # тело генератора
    
    yield result
    
    # конец тела генератора
```

**Функции-генераторы** – обычные функции, в которых вместо слова `return` используется ключевое слово `yield`.  
При этом при вызове функции вы получаете объект-генератор, реализующий внутреннюю логику функции.  

Каждый раз, когда выполняется `yield`, объект сохраняет свой контекст выполнения (значения всех переменных, стек вызовов и т.д.)  
При следующем вызове `next` к этому объекту контекст восстанавливается и объект продолжает выполняться с этой же точки  

In [2]:
# попробуем реализовать бесконечный счетчик count из модуля itertools
def count(start, step):
    counter = start
    while True:
        yield counter
        counter += step

In [4]:
my_gen_func = count(100, 10)
for i in range(10):
    print(next(my_gen_func))  # с функцией генератором также работает next

100
110
120
130
140
150
160
170
180
190


In [5]:
# функция генератор уже является итератором
id(my_gen_func) == id(iter(my_gen_func))

True

---
Как и все генераторы, функция-генератор останавливается с вызовом `StopIteration`.  

Это происходит при следующих условиях:
1. Интерпретатор достиг конца выполнения функции и не встретил никаких инструкций
- В функции был выполнен return
- «Вручную» вызвано неперехваченное исключение `StopIteration`

In [6]:
def first_gen(input_):
    yield input_
    input_ += 1
    print(input_)
    
    
my_first_gen = first_gen(5)

print(next(my_first_gen))

5


In [7]:
# StopIteration
next(my_first_gen)  # print из тела генератор

6


StopIteration: 

In [ ]:
def second_gen(input_):
    yield input_
    input_ += 1
    
    yield input_
    input_ += 1
    
    return input_
    
    
my_second_gen = second_gen(10)

print(next(my_second_gen))
print(next(my_second_gen))
print(next(my_second_gen))

In [ ]:
def last_gen():
    for i in range(10):
        yield i
        if i == 5:
            raise StopIteration
    
my_last_gen = last_gen()

for _ in range(10):
    print(next(my_last_gen))

In [ ]:
# for перехватывает ваше исключение
my_last_gen = last_gen()
for i in my_last_gen:
    print(i)

---
Разберем ещё один пример:

In [8]:
def my_animal_generator():
    yield 'корова'
    print('---')
    for animal in ['кот', 'собака', 'медведь']:
        yield animal
    print('---')
    yield 'кит'

a = my_animal_generator()
print(next(a))
# print('---') вызван не будет

корова


In [9]:
print(next(a))

for i in a:
    print(i)

---
кот
собака
медведь
---
кит


---
Попробуем реализивать генератор чисел Фибоначчи. 
Последовательность чисел Фибоначчи представляет собой:  
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711 ...

А общая формула имеет следующий вид:  

$$F_0=0,  F_1=1$$  

$$F_n=F_{n-1}+F_{n-2}, n\geq\ 2$$

In [10]:
def fib():
    a, b = 0, 1
    yield a  # F0
    yield b  # F1

    while True:
        a, b = b, a + b
        yield b

In [11]:
for num in fib():
    print(num)
    if num > 10000:
        break

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946


С помощью метода `close()` можно закрыть генератор  

In [13]:
fib_gen = fib()
fib_gen.close()  # закрыли генератор

for i in fib_gen:  # цикл не выполнится ни одного раза
    print(i)
    
fib_gen

<generator object fib at 0x7fd3f8a5f990>

Итак, с помощью функции-генератора мы с вами написали эффективную реализацию последовательности чисел Фибоначчи, которой для вычисления необходимо хранить в памяти только два предыдущих числа, а вычисления происходят только в момент вызова функции-генератора. 

Функции генераторы применяются в тех случаях, когда вам:
- необходимо сэкономить память при работе с массивными структурами (ленивые вычисления)
- удобна логика работы генератора, внутри которого можно описать сложный процесс выбора

Пример, который вам может пригодиться – построчное чтение 5Гб файла  

Примеры модулей:
- `keras` – генератор данных для обучения
- `faker` – генерация фейковых данных для заполнения и тестирования объектов
- `gmp2` – генерация простых чисел




---
## Корутины (Coroutines)

В языке Python генераторы могут использоваться и в «обратную сторону» – принимая в себя внутрь значения.  
До этого мы только получали значения из наших генераторов, посмотрим теперь, как сделать так, чтобы можно было передавать информацию внутрь нашего генератора.

Попробуем сделать, корутину, которая будет принимать некий паттерн, проверять его наличие в какой-либо строке и подсчитывать количество вхождений.

In [14]:
def grep(pattern):
    i = 0
    while True:
        line = yield  # отличие от генератора
        if pattern in line:
            i += 1
            print(f"Found! Count: {i}")

Отличие в синтаксисе только в том, что теперь слева от `yield` стоит переменная, которая будет принимать значение, которое мы передаем из вне.

In [15]:
g = grep("python")  # создаем объект корутину, которой в качетве аргумента передаем паттерн

Единственная особенность в том, чтобы запустить корутину, её нужно проинициализировать.
Делается это с помощью команды `next(coroutine)` либо `coroutine.send(None)`, что является одним и тем же

$$next(coroutine) \equiv coroutine.send(None)$$

Ранее, когда мы делали команду `next()` к нашему генератору, мы делали ему `send(None)`

In [16]:
# инициализируем корутину
g.send(None)  # либо next(g)

In [17]:
# отправим нашей корутине, новое значение для проверки вхождения в строку
g.send("some string here python")

Found! Count: 1


In [18]:
g.send("anouther string")

In [19]:
g.send("python python!!!")

Found! Count: 2


In [20]:
# чтобы закрыть корутину можно использовать метод close
g.close()

Попробуем прислать нашей корутине, первым действием что-то отличное от `None`

In [22]:
g = grep("new_pattern")

g.send("New string with new_pattern")

TypeError: can't send non-None value to a just-started generator

И получим соответсвующую ошибку, что только что стартовавшему генератору, нельзя присылать значение отличное от `None`

---
Рассмотрим момент, который окончательно "вынесет мозг". Это когда корутина может одновременно принимать и отправлять данные.  

Давайте сделаем генератор, который будет возвращать нам бесконечную арифметическую прогрессию, то мы сможем переопределять начальные значения после запуска генератора.

In [ ]:
def counter(start, step):
    n = start
    while True:
        input_data = yield n
        if input_data is not None:
            n = input_data[0]
            step = input_data[1]
        else:
            n += step

In [ ]:
c = counter(100, 10)  # создаем корутину с начальными значениями
for _ in range(10):
    print(c.send(None))

In [ ]:
# передаем корутине новые значения старта и шага прогресии без создания новой корутины
print(c.send((500, 1)))  

In [ ]:
for _ in range(10):
    print(c.send(None))
    
c.close()

---
### Пример использования:
Живой пример использования данной идеи – модуль tornado
Асинхронная сетевая библиотека для обработки запросов
скалируется до десятков тысяч одновременно открытых соединений
построен на генераторах и корутинах (с кучей дополнительного, конечно же)

Разница между Django и Tornado на задачах вида «загрузи ответ от другого сервера и верни мне данные» – до 10 раз в пользу торнадо (и других асинхронных фреймворков) [источник](https://klen.github.io/python-web-benchmarks.html)

---
### Подведем итоги по генераторам и корутинам:
В более общем смысле, генераторы и корутины – это объекты, которые умеют получать управление, выполнять некоторую работу и отдавать управление обратно.  
Данная логика работы называется «кооперативная многозадачность». Грамотно реализуя преимущества данной логики, можно построить на языке Python приложения, обрабатывающие десятки тысяч запросов ежесекундно.

Более подробно с многозачачностями можно познакомиться [тут](https://kvckr.github.io/mag/sp/7.html)